### Installing Required Packages and Setting PrettyTable Default

In [2]:
!pip install ipython-sql sqlalchemy

import prettytable
prettytable.DEFAULT = 'DEFAULT'

### Importing Libraries and Connecting to SQLite Database

In [4]:
import csv, sqlite3

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()  # Cursor object to execute SQL queries

### Installing and Loading ipython-sql Extension for Jupyter

In [ ]:
!pip install ipython-sql

# Loads %sql and %%sql magics
%load_ext sql

### Importing Pandas and Connecting to SQLite Database

In [ ]:
import pandas as pd
import sqlite3

# Establishes a connection to the 'FinalDB.db' SQLite database
conn = sqlite3.connect("FinalDB.db")

In [ ]:
df = pd.read_excel('synthetic_cc_tx.xlsx')

In [ ]:
df.to_sql("CC_DATA", conn, if_exists="replace", index=False)

In [ ]:
# Close the connection
conn.close()

print("Data successfully loaded into FinalDB.db")

In [ ]:
#Establishing a connection between SQL magic module and the database `FinalDB.db`
%reload_ext sql
%sql sqlite:///FinalDB.db

### Checking loaded tables

In [ ]:
%%sql
-- Check tables loaded in the database
SELECT name FROM sqlite_master WHERE type='table';

In [ ]:
%%sql
PRAGMA table_info(CC_DATA);

In [ ]:
df.head()

In [ ]:
## Calculate: total_transactions, unique_users and total_transaction_amount for every date and hour combination.

In [ ]:
%%sql
SELECT 
    DATE(timestamp) AS txn_date,
    STRFTIME('%H', timestamp) AS txn_hour,
    COUNT(*) AS total_transactions,
    COUNT(DISTINCT user_id) AS unique_users,
    SUM(tx_amt) AS total_transaction_amount
FROM CC_DATA
GROUP BY txn_date, txn_hour
ORDER BY txn_date, txn_hour;

In [ ]:
#timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
#columns for date and hour
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour

#Group by date and hour
result = df.groupby(['date', 'hour']).agg(
    total_transactions=('tx_id', 'count'),
    unique_users=('user_id', 'nunique'),
    total_transaction_amount=('tx_amt', 'sum')).reset_index()
#Display result
print(result)

## Insights:
#### 1. User Engagement by Time of Day
Hourly transaction counts help identify peak hours of user activity.

Businesses can Optimize marketing efforts during high-traffic hours.

Allocate server or delivery resources efficiently.

Run promotions or offers during low-engagement hours to boost usage.

#### 2. Transaction Volume Patterns
total_transactions shows how active the platform is.

If consistent spikes are observed at specific times (e.g., evenings), it reflects user behavior trends.

Sharp drops may indicate technical issues, off-hours, or lack of engagement.

#### 3. Unique Users vs. Transactions
unique_users vs. total_transactions comparison helps identify:

Repeat transactions by the same user in a short time span.

One-time users, indicating low retention or low engagement.

High repeat usage can signal customer loyalty, while one-off transactions may need user retention strategies.

#### 4. Revenue Analysis
total_transaction_amount reflects the monetary value of transactions in each time window.

High-value transactions in off-peak hours might be due to:

Bulk/wholesale purchases

Loyal or high-spending customers

Businesses can analyze these to:

Identify and segment high-value users

Tailor offers or services for them

#### 5. Temporal Trends
Looking at how these metrics evolve across days and hours allows:

Detection of daily/weekly cycles (e.g., weekends vs weekdays)

Planning operations, support, and marketing schedules accordingly

#### 6. Fraud or Anomaly Detection
Unusual spikes in transaction count or amount (especially from a few users) can trigger fraud checks.

Outlier detection (e.g., very high tx_amt in low activity hours) is critical in financial systems.

#### 7. Load Forecasting
Understanding hourly demand helps:

Predict system/server load

Prepare for scale (e.g., sale events, paydays, festivals)

In [ ]:
##  Calculate hour with highest transaction_amount for every date

In [ ]:
%%sql
SELECT DATE(timestamp) AS transaction_date,STRFTIME('%H', timestamp) AS transaction_hour,SUM(tx_amt) AS total_amount
FROM CC_DATA
GROUP BY transaction_date, transaction_hour
HAVING total_amount = (SELECT MAX(hour_total) FROM (SELECT SUM(tx_amt) AS hour_total FROM CC_DATA WHERE DATE(timestamp) = transaction_date GROUP BY STRFTIME('%H', timestamp)))
ORDER BY transaction_date;


In [ ]:
grouped = df.groupby(['date', 'hour'])['tx_amt'].sum().reset_index()
#hour with highest transaction amount per date
result = grouped.loc[grouped.groupby('date')['tx_amt'].idxmax()].reset_index(drop=True)
print(result)

#### Peak Earning Hours Per Day
On each day, a specific hour contributes the most revenue.

Businesses can:

Target promotions during this time to further capitalize.

Ensure system uptime and smooth operations during those hours.

Allocate resources or support teams accordingly.

#### Revenue-Driving Patterns
Example: Jan 1st saw the highest revenue during 9 PM (21 hr) — indicates evening spending behavior.

Jan 2nd’s peak at 5 AM might signal:

Users from different time zones.

Scheduled transactions.

Automated systems.

Jan 3rd at 8 AM might reflect morning use (e.g., payments, checkouts).

#### Time-Based Segmentation
Different users may engage at different times, so analyzing high-transaction hours helps create time-specific user personas:

Night owls vs early risers.

Work-hour spenders vs off-hour shoppers.

#### Anomaly or Fraud Check
Unusual spikes in non-business hours (e.g., 5 AM) should be investigated.

Repeated peaks in odd hours might point to bots, automated scripts, or testing data if synthetic.

#### Use Case for Dynamic Pricing / Load Management
Dynamic pricing can be introduced during peak hours.

Predictive modeling can use this trend to forecast peak loads and preemptively allocate infrastructure.